## 가상환경 만들기

In [ ]:
!conda create -n llm_img python=3.10 -y
!conda activate llm_img
!pip install openai pillow ipykernel python-dotenv  matplotlib opencv-python

In [24]:
import base64
from PIL import Image
from io import BytesIO
from openai import OpenAI
import os
from dotenv import load_dotenv

# 한글로 주석 표기하기
load_dotenv()      # 환경변수 로드
api_key = os.getenv("OPENAI_API_KEY")   # OpenAI API 키를 환경변수에서 가져오기
print(api_key)


sk-proj-6OL30nlPvKM5pvcPc2PLMbjaE3JyVKfPff83lFXRqI-YFZqk4gheOPOlTNsKubQDQ3MlPhRMdgT3BlbkFJfM9HoXfnoLjCXFG3sqzYMXwo0xXzCj24w-vCSkIbUJflm8gvi-9VLmLmwFGaKctKAfLvkE8A


In [25]:
# 이미지를 base64 문자열로 인코딩하는 함수
def encode_image(image):
    buffered = BytesIO()   # 메모리 버퍼 생성
    w, h = image.size    # 이미지의 가로(w), 세로(h) 크기 가져오기
    if w > 512 or h > 512:
        scale = 512 / max(w, h)  # 큰 쪽 기준으로 512 크기에 맞춰 비율 계산
    else:
        scale = 1.0  # 크기 조정이 필요 없으면 비율 1.0
    resize_im = image.resize((int(w * scale), int(h * scale)))  # 비율 유지하면서 이미지 크기 조정
    resize_im.save(buffered, format="JPEG")  # JPEG 포맷으로 메모리에 저장
    img_str = base64.b64encode(buffered.getvalue()).decode('utf-8')  # base64 문자열로 인코딩
    return img_str

In [26]:
# GPT 모델에게 이미지와 프롬프트를 보내 결과를 받아오는 함수
def img_classification(prompt, image):
    client = OpenAI(api_key=api_key)   # OpenAI 클라이언트 생성

    # 이미지와 프롬프트를 포함한 메시지를 GPT에 전달
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64, {encode_image(image)}"                   # base64 인코딩된 이미지 URL 
                        }
                    },
                    {
                        "type": "text" ,   # 프롬프트
                        "text": prompt    
                    }
                ]
            }
        ]
    )
    return response.choices[0].message.content     # GPT의 응답 내용 반환


In [28]:
# GPT에게 보낼 프롬프트 정의
prompt = f"""
영상을 보고 다음 보기 내용이 포함되면 1, 포함되지 않으면 0으로 분류해줘.
보기 = [건축물, 바다, 산]
JSON format으로 키는 'building', 'sea', 'mountain'으로 하고 각각 건축물, 바다, 산에 대응되도록 출력해줘.
자연 이외의 건축물이 조금이라도 존재하면 'building'을 1로, 물이 조금이라도 존재하면 'sea'을 1로, 산이 조금이라도 보이면 'mountain'을 1로 설정해줘.
markdown format은 포함하지 말아줘.
"""

# 'imgs_classification' 폴더에서 .jpg 파일만 리스트로 가져오기
files = [f for f in os.listdir('imgs_classification') if f.endswith('jpg')]

# 파일 이름 순으로 정렬하여 하나씩 처리
for f in sorted(files):
    img = Image.open(os.path.join('imgs_classification', f))  # 이미지 열기
    response = img_classification(prompt, img)  # GPT로부터 분류 결과 받기
    display(img)  # 이미지 출력
    print(response)  # 결과 출력

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-******************************************************************************************************************************************************kE8A. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}